In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

df_reviews = pd.read_csv("../data/movie_reviews_processed.csv")

training_data, testing_data = train_test_split(
    df_reviews, 
    test_size = 0.25, 
    random_state = 123
)

In [2]:
#| label: python-regression-model-train
import statsmodels.api as sm
import statsmodels.formula.api as smf

# features = ["review_year_0", "release_year_0",
#   "age_sc", "length_minutes_sc", 
#   "total_reviews_sc", "word_count_sc", 
#   "genre", "gender", 
#   "reviewer_type", "work_status", 
#   "season"]

model_train_reg = smf.ols(
    formula = 'rating ~ review_year_0 \
    + release_year_0 \
    + age_sc \
    + length_minutes_sc \
    + total_reviews_sc \
    + word_count_sc \
    + genre'
    ,
    data = training_data
).fit()

In [3]:
model_train_reg.model.exog

array([[ 1.        ,  0.        ,  0.        , ...,  0.78062224,
         0.72139957,  0.33250418],
       [ 1.        ,  0.        ,  0.        , ...,  0.95413241,
        -0.8184751 ,  2.89629682],
       [ 1.        ,  0.        ,  0.        , ...,  1.04088749,
        -0.56969901,  2.699082  ],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.17368368,
         1.31550225, -0.25914027],
       [ 1.        ,  0.        ,  1.        , ..., -1.56176501,
        -0.31140882,  0.13528937],
       [ 1.        ,  0.        ,  0.        , ...,  1.99519341,
        -0.31070407,  1.31857827]])

In [4]:
model_train_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.484
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     53.17
Date:                Sun, 10 Mar 2024   Prob (F-statistic):           1.29e-96
Time:                        17:31:21   Log-Likelihood:                -458.36
No. Observations:                 750   AIC:                             944.7
Df Residuals:                     736   BIC:                             1009.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.4132      0.079     30.634      0.000       2.259       2.568
genre[T.Comedy]       0.5201      0.065      8.033      0.000       0.393       0.647
genre[T.Drama]        0.5655      0.044     12.818      0.000       0.479       0.652
genre[T.Horror]       0.0069      0.078      0.089      0.929      -0.146       0.160
genre[T.Kids]        -0.0373      0.065     -0.571      0.568      -0.165       0.091
genre[T.Other]        0.0075      0.073      0.103      0.918      -0.136       0.151
genre[T.Romance]      0.0236      0.070      0.337      0.736      -0.114       0.161
genre[T.Sci-Fi]      -0.0173      0.079     -0.220      0.826      -0.172       0.137
review_year_0         0.0096      0.003      2.957      0.003       0.003       0.016
release_year_0        0.0110      0.002      5.874      0.000       0.007       0.015
age_sc               -0.0428      0.017     -2.551      0.011      -0.076      -0.010
length_minutes_sc     0.1625      0.018      8.853      0.000       0.126       0.199
total_reviews_sc      0.2513      0.018     13.976      0.000       0.216       0.287
word_count_sc        -0.1148      0.017     -6.572      0.000      -0.149      -0.080
==============================================================================
Omnibus:                       13.448   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.864
Skew:                          -0.237   Prob(JB):                     0.000359
Kurtosis:                       3.532   Cond. No.                         211.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
import dalex as dx
import matplotlib.pyplot as plt

features = [
    "review_year_0", 
    "release_year_0",
    "age_sc", 
    "length_minutes_sc", 
    "total_reviews_sc", 
    "word_count_sc", 
    # "genre", 
    ]

X = training_data[features]

X[X.select_dtypes(include='object').columns] = X.select_dtypes(include='object').astype('category')

train_explain = dx.Explainer(
    model_train_reg, 
    data = X, 
    y = training_data.rating,
    verbose = False
)

train_performance = train_explain.model_performance()

perf_plot = train_performance.plot()


/Users/micl/anaconda3/envs/book-of-models/lib/python3.12/site-packages/dalex/_explainer/object.py:138: UserWarning: 
  -> predicted values  : 'predict_function' returns an Error when executed: 
predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'genre' is not defined
    rating ~ review_year_0     + release_year_0     + age_sc     + length_minutes_sc     + total_reviews_sc     + word_count_sc     + genre
                                                                                                                                      ^^^^^
  checks.check_predict_function_and_model_type(predict_function, model_type,
/Users/micl/anaconda3/envs/book-of-models/lib/python3.12/site-packages/dalex/_explainer/object.py:138: UserWarning: 
  -> predicted values  : 'predict_function' must return numpy.ndarray (1d)
  checks.check_predict_function_

predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'genre' is not defined
    rating ~ review_year_0     + release_year_0     + age_sc     + length_minutes_sc     + total_reviews_sc     + word_count_sc     + genre
                                                                                                                                      ^^^^^


PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'genre' is not defined
    rating ~ review_year_0     + release_year_0     + age_sc     + length_minutes_sc     + total_reviews_sc     + word_count_sc     + genre
                                                                                                                                      ^^^^^

In [ ]:
break_down_plot = train_explain.predict_parts(
    new_observation = X.loc[1,features]
)

In [ ]:
break_down_plot.plot(

)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

features = [
    "review_year_0", 
    "release_year_0",
    "age_sc", 
    "length_minutes_sc", 
    "total_reviews_sc", 
    "word_count_sc", 
    "genre", 
    ]

model =  'rating_good ~ ' + " + ".join(features)

model_train_class = smf.glm(
    formula = model,
    data = training_data,
    family = sm.families.Binomial()
).fit(maxiter= 2000, tol = 1e-12)

model_train_class.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            rating_good   No. Observations:                  750
Model:                            GLM   Df Residuals:                      736
Model Family:                Binomial   Df Model:                           13
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -367.76
Date:                Thu, 08 Feb 2024   Deviance:                       735.52
Time:                        20:44:46   Pearson chi2:                     672.
No. Iterations:                     6   Pseudo R-squ. (CS):             0.3189
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.7239      0.439     -3.931      0.000      -2.583      -0.864
genre[T.Comedy]       2.4492      0.448      5.462      0.000       1.570       3.328
genre[T.Drama]        1.9952      0.256      7.789      0.000       1.493       2.497
genre[T.Horror]      -0.0215      0.397     -0.054      0.957      -0.799       0.756
genre[T.Kids]        -0.1908      0.358     -0.533      0.594      -0.893       0.511
genre[T.Other]        0.0147      0.363      0.041      0.968      -0.696       0.726
genre[T.Romance]      0.1751      0.385      0.455      0.649      -0.579       0.929
genre[T.Sci-Fi]       0.2399      0.416      0.577      0.564      -0.575       1.055
review_year_0         0.0272      0.018      1.529      0.126      -0.008       0.062
release_year_0        0.0355      0.011      3.370      0.001       0.015       0.056
age_sc               -0.2167      0.094     -2.296      0.022      -0.402      -0.032
length_minutes_sc     0.5661      0.106      5.360      0.000       0.359       0.773
total_reviews_sc      0.9022      0.109      8.312      0.000       0.689       1.115
word_count_sc        -0.5834      0.106     -5.522      0.000      -0.790      -0.376
=====================================================================================
"""

In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model =  'rating_good ~ ' + " + ".join(features)

model_train_class = smf.glm(
    formula = model,
    data = training_data,
    family = sm.families.Binomial()
).fit()


model_train_class.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            rating_good   No. Observations:                  750
Model:                            GLM   Df Residuals:                      743
Model Family:                Binomial   Df Model:                            6
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -425.64
Date:                Sun, 10 Mar 2024   Deviance:                       851.29
Time:                        17:32:40   Pearson chi2:                     750.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.2052
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.0450      0.361     -2.893      0.004      -1.753      -0.337
review_year_0         0.0218      0.016      1.337      0.181      -0.010       0.054
release_year_0        0.0418      0.009      4.497      0.000       0.024       0.060
age_sc               -0.1363      0.084     -1.615      0.106      -0.302       0.029
length_minutes_sc     0.5261      0.092      5.693      0.000       0.345       0.707
total_reviews_sc      0.6006      0.089      6.748      0.000       0.426       0.775
word_count_sc        -0.6498      0.097     -6.719      0.000      -0.839      -0.460
=====================================================================================
"""

In [10]:
from pycm import ConfusionMatrix

predictions = model_train_class.predict(testing_data)
predictions = np.where(predictions >= .5, 1, 0)

predictions = pd.Series(predictions)

cm = ConfusionMatrix(
    testing_data.rating_good.to_numpy(), 
    predictions.to_numpy(), 
    digit = 3
)

# cm.overal_stat # lots of stats!

stats_ = [
    cm.overall_stat[stat] 
    for stat in ['Overall ACC', 'TPR Macro', 'TNR Macro', 'PPV Macro', 'NPV Macro']
]

pd.DataFrame({
    'Metric': ['Accuracy', 'Sensitivity', 'Specificity',  
               'Precision', 'Negative Predictive Value'],
    'Value': stats_
})

,Metric,Value
0,Accuracy,0.708000
1,Sensitivity,0.699859
2,Specificity,0.699859
3,Precision,0.715972
4,Negative Predictive Value,0.715972


In [ ]:
from marginaleffects import avg_slopes, comparisons

# avg_slopes(model_train_class, variables = "word_count_sc")
comparisons(model_train_class, variables = {"word_count_sc": 'sd'})

TypeError: cannot call `map_groups` when grouping by an expression